First we include some helper functions to interact with OpenAI's API. 

In [ ]:
from src.config import client, model
from src.utils import load_config
from src.assistants.analyst.literature import literature_search
import json

def create_message_and_run_analyst_module(assistant, instructions, user_prompt, thread = None, save_tool_request_path = None):
    '''
    This function is a slightly simplified version of how the analyst module would be in WildfireGPT.
    In this experiment, we only seek to understand how the changes in user profile affect the retrieved literature and the recommendations.
    '''
    if thread is None:
        thread = client.beta.threads.create()
    client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=user_prompt
    )
    run = client.beta.threads.runs.create_and_poll(
        thread_id=thread.id,
        assistant_id=assistant.id,
        instructions=instructions,
        parallel_tool_calls=False
    )

    tool_outputs = []

    if run.required_action is not None:
        # Loop through each tool in the required action section
        for tool in run.required_action.submit_tool_outputs.tool_calls:
            if tool.function.name == "literature_search":
                function_args = json.loads(tool.function.arguments)
                print(function_args)
                if save_tool_request_path is not None:
                    with open(save_tool_request_path, "w") as f:
                        f.write(json.dumps(function_args))
                function_response = literature_search(**function_args)
                path = 'src/assistants/analyst/appendix/literature.md'
                with open(path, "r") as f:
                    appendix = f.read()
                    function_response += appendix
                tool_outputs.append({
                    "tool_call_id": tool.id,
                    "output": function_response
                })

        if len(tool_outputs) > 0:
            run = client.beta.threads.runs.submit_tool_outputs(
                thread_id=thread.id,
                run_id=run.id,
                tool_outputs=tool_outputs,
            )

    return thread
    

def generate_plan(profile):
    '''
    This function is a slightly simplified version of how the planning module would be in WildfireGPT.
    In this experiment, we only seek to understand how the changes in user profile affect the retrieved literature and the recommendations.
    '''
    plan_config = load_config("src/assistants/plan/config.yml")
    plan_instruction = f"{plan_config['instructions']}\n{plan_config['available_datasets']}\n{plan_config['example']}\nHere is the information about your client: {profile}"
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": plan_instruction},
            {"role": "user", "content": "Simply output the plan and nothing else."},
        ]
    )
    return completion.choices[0].message.content


# There are five different user profiles that we will be using in this ablation experiment. 


template_profile = '''- **Profession:** {profession} in Virginia.
- **Concern:** Managing the forest, keeping it healthy, while {concern}, and protecting properties from potential wildfires.
- **Location:** Near Covington, VA with Latitude 37.7935 and Longitude -79.9939.
- **Time:** Recommendations to be implemented within the next 5 to 10 years.
- **Scope:** Management of the forest and properties to maximize {scope}, and protect against potential wildfires.
'''

profiles = {
    "homeowner": {
        "profession": "Homeowner",
        "concern": "maximizing marketable species",
        "scope": "health and marketable species",
    },
    "civil_engineer": 
    {
        "profession": "Civil Engineer",
        "concern": "ensuring structural and infrastructural resilience",
        "scope": "drainage efficiency and slope stability",
    },
    "ecologist": 
    {
        "profession": "Ecologist",
        "concern": "maintaining biodiversity and ecosystem services",
        "scope": "ecological resilience and habitat connectivity",
    },
    "emergency_manager": 
    {
        "profession": "Emergency Manager",
        "concern": "establishing defendable space and evacuation corridors",
        "scope": "emergency access and response capabilities",
    },
    "firefighter": 
    {
        "profession": "Firefighter",
        "concern": "establishing fuel breaks and maintaining tactical advantages",
        "scope": "suppression effectiveness and crew safety",
    },
}

# Below was the original instruction to the Analyst Module from the case study `case_studies/Private Property Protection`. 

In the ablation study below, we modify the user profile slightly to see how the model's predictions change. Specifically, we try to observe changes in 
- the plan proposed by the planning module with the updated user profile
- the content of the literature review generated by the analyst module with the updated user profile and plan
- the content of the recommendation generated by the analyst module with the updated user profile, plan, and literature review

There are five different user profiles that we will be using in this ablation experiment. They are: homeowner, civil_engineer, ecologist, emergency_manager, and firefighter. You can see the minimal level of changes made to the user profile in the code block above.

We will save everything under the folder `ablation_study` in the current working directory for reference and comparison.

In [ ]:
thread_messages = client.beta.threads.messages.list("thread_KKbnKKvkPOFivz8r5ZAYPUCn")
assistant = client.beta.assistants.retrieve(thread_messages.data[0].assistant_id)
print(assistant.instructions)

In [ ]:
# Now set the identify of the user
user_identity = "civil_engineer"

def generate_profile(profile):
    profile_text = template_profile.format(**profile)
    return profile_text

profile = generate_profile(profiles[user_identity])
print(profile)

In [ ]:
plan = generate_plan(profile)
print(plan)

import os
if not os.path.exists(f"ablation_study/{user_identity}"):
    os.makedirs(f"ablation_study/{user_identity}")

with open(f"ablation_study/{user_identity}/plan.md", "w") as f:
    f.write(plan)

In [6]:
literature_search_user_prompt = "Let's review some relevant literature."

recommendation_user_prompt = "Develop recommendations for species selection and planting strategies to enhance forest resilience."

analyst_instruction = "Here is the information about your client:\n\n{profile}\n\nHere is your overall plan to assist your client: {plan}\n\nHere is your client's last message: {user_prompt}\n\nHere is your plan for this step: {this_step}\n\nIf you give any recommendations, please provide the reasoning behind them and suggest the client to ask for supporting scientific evidence."

this_step = {
    'literature': "Respond to the client's questions. Analyze literature. `literature_search`", 
    'recommendation': "Respond to the client's questions. Develop recommendations. `no tool needed`"
}

In [ ]:
instrucion = analyst_instruction.format(profile=generate_profile(profiles["ecologist"]), plan=plan, user_prompt=literature_search_user_prompt, this_step=this_step['literature'])

thread = create_message_and_run_analyst_module(assistant, instrucion, literature_search_user_prompt, save_tool_request_path = f"ablation_study/{user_identity}/literature_search_query.txt")

In [ ]:
print(thread.id)
# get last message from the thread
thread_messages = client.beta.threads.messages.list(thread.id)

In [ ]:
print(thread_messages.data[0].content[0].text.value)

# save the thread id to a file
with open(f"ablation_study/{user_identity}/thread_id.txt", "w") as f:
    f.write(thread.id)

with open(f"ablation_study/{user_identity}/literature_search_result.md", "w") as f:
    f.write(thread_messages.data[0].content[0].text.value)

In [ ]:
instrucion = analyst_instruction.format(profile=generate_profile(profiles["ecologist"]), plan=plan, user_prompt=recommendation_user_prompt, this_step=this_step['recommendation'])

thread = create_message_and_run_analyst_module(assistant, instrucion, recommendation_user_prompt, thread)

In [ ]:
print(thread.id)
# get last message from the thread
thread_messages = client.beta.threads.messages.list(thread.id)
print(thread_messages.data[0].content[0].text.value)

with open(f"ablation_study/{user_identity}/recommendation.md", "w") as f:
    f.write(thread_messages.data[0].content[0].text.value)